In [ ]:
import sys
!cp ../input/rapids/rapids.0.17.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd

import cuml
from cuml.neighbors import KNeighborsClassifier
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

from catboost import CatBoostRegressor

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv',index_col=0)
test = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv',index_col=0)
submission = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')

In [ ]:
x = train.drop(['target'], axis=1).copy()
y = train['target'].copy()

In [ ]:
# Get OOF training preds for GM

kf = KFold()
x['gm'] = 0

for idx_train, idx_valid in kf.split(x):
    xtrain, xvalid = x.iloc[idx_train], x.iloc[idx_valid]
    ytrain, yvalid = y.iloc[idx_train], y.iloc[idx_valid]
    
    # Train Gaussian Mixture
    gm_y = ytrain.values.reshape(-1,1)
    gm = GaussianMixture(n_components=2, random_state=0).fit(gm_y)
    gm_preds = gm.predict(gm_y)
    
    # Get Gaussian Mixture for test set by using knnclassifier
    knn = KNeighborsClassifier(n_neighbors = 7)
    knn.fit(xtrain.values, gm_preds)
    knn_preds = knn.predict(xvalid.values)
    x['gm'].iloc[idx_valid] = knn_preds

In [ ]:
# Get GM preds for test set

# Get Gaussian Mixture for test set by using knnclassifier
knn_x = x.drop(['gm'], axis=1).copy()
knn_y = x['gm'].copy()

knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(knn_x, knn_y)
knn_preds = knn.predict(test)

test['gm'] = knn_preds

In [ ]:
model = LinearRegression()
model = model.fit(x, y)
preds = model.predict(test)

submission['target'] = preds
submission.to_csv('submission.csv', index=False)